# Requirements

In [ ]:
%pip install transformers
%pip install torch
%pip install pickle5
%pip install mpld3
%pip install scikit-learn

In [ ]:
import torch
# torch.cuda.empty_cache()

# Debias Model

In [ ]:
# Use "bert", "1. bert" if not using a debiased model
# debiased_model = "bert"
# debiased_folder = "1. bert"
# image_caption = "Bert"

# debiased_model = "sent_debiased"
# debiased_folder = "2. sent_debiased"
# image_caption = "Sent Debiased"

# debiased_model = "contextualised"
# debiased_folder = "3. contextualised"
# image_caption = "Contextualised"

debiased_model = "cds"
debiased_folder = "4. cds"
image_caption = "CDS"

prefix = "../data/extracted"

is_sentence = True
word_or_sent = "word" if not is_sentence else "sentence"

# Load words and embeddings

In [ ]:

import pickle5 as pickle

def open_pklfile(filepath, size):
	"""Open pickle file from the file path.

	Load all contents if `size` is 0, else slice and return.
	"""
	with open(filepath, "rb") as f:
		data = pickle.load(f)
	return data if not size else data[:size]


# Experiment 1 : Clustering

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import mpld3
from cycler import cycler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

mpld3.enable_notebook()
mpl.rc("savefig", dpi=200)
mpl.rcParams['figure.figsize'] = (8,8)
mpl.rcParams['axes.prop_cycle'] = cycler(color='rc')


def visualize(embeddings, labels, ax, title, random_state, num_clusters = 2):
    X_embedded = TSNE(n_components=2, random_state=random_state).fit_transform(embeddings)
    if num_clusters == 2:
        for x, l in zip(X_embedded, labels):
            if l:
                ax.scatter(x[0], x[1], marker = '.', c = 'c')
            else:
                ax.scatter(x[0], x[1], marker = 'x', c = 'darkviolet')
    else:
        ax.scatter(X_embedded[:,0], X_embedded[:,1], c = labels)
    ax.text(.01, .9, title ,transform=ax.transAxes, fontsize=18)


def cluster_and_visualize(embed_bef, embed_aft, labels, save_filename, random_state = 1, num=2):
    fig, axs = plt.subplots(1, 2, figsize=(15, 3))

    # Clustering embeddings before debiasing
    prediction_bef = KMeans(n_clusters=num, random_state=random_state).fit_predict(embed_bef.cpu())
    visualize(embed_bef.cpu(), labels, axs[0], "Bert", random_state)
    correct = [1 if item1 == item2 else 0 for (item1, item2) in zip(labels, prediction_bef)]
    print("Precision before: ", sum(correct)/len(correct))
    
    # Clustering embeddings after debiasing
    prediction_aft = KMeans(n_clusters=num, random_state=random_state).fit_predict(embed_aft.cpu())
    visualize(embed_aft.cpu(), labels, axs[1], image_caption, random_state)
    correct = [1 if item1 == item2 else 0 for (item1, item2) in zip(labels, prediction_aft)]
    print("Precision after: ", sum(correct)/len(correct))
    
    fig.show()
    fig.savefig("../results/" + save_filename, bbox_inches='tight')


### 2016 Hard_Debiased Clustering

In [ ]:
# 2016 Hard_debiased
# Cluster most biased words before and after debiasing

# debiased model with bert_words
debiased_top500_male_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_2016_male_2500_embeddings.pkl", 500)
debiased_top500_female_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_2016_female_2500_embeddings.pkl", 500)

# debiased model with debiased_words
hong_debiased_top500_male_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_hong2016_male_2500_embeddings.pkl", 500)
hong_debiased_top500_female_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_hong2016_female_2500_embeddings.pkl", 500)

embeddings = torch.cat([debiased_top500_male_embeddings, debiased_top500_female_embeddings], dim=0)
debiased_embeddings = torch.cat([hong_debiased_top500_male_embeddings, hong_debiased_top500_female_embeddings], dim=0)

labels = [0] * 500 + [1] * 500  # 0 for male, 1 for female
cluster_and_visualize(embeddings, debiased_embeddings, labels, f"{debiased_folder}/{debiased_model}_{word_or_sent}_clustering_2016")


### 2018 GN_GloVe Clustering

In [ ]:
# 2018 GN_GloVe
# Cluster most biased words before and after debiasing

# Bert
debiased_top500_male_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_2018_male_2500_embeddings.pkl", 500)
debiased_top500_female_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_2018_female_2500_embeddings.pkl", 500)

# Debiased
hong_debiased_top500_male_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_hong2018_male_2500_embeddings.pkl", 500)
hong_debiased_top500_female_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_hong2018_female_2500_embeddings.pkl", 500)

embeddings = torch.cat([debiased_top500_male_embeddings, debiased_top500_female_embeddings], dim=0)
debiased_embeddings = torch.cat([hong_debiased_top500_male_embeddings, hong_debiased_top500_female_embeddings], dim=0)

labels = [0] * 500 + [1] * 500  # 0 for male, 1 for female
cluster_and_visualize(embeddings, debiased_embeddings, labels, f"{debiased_folder}/{debiased_model}_{word_or_sent}_hongclustering_2018")


In [ ]:
from sklearn import svm
import random
random.seed(10)

def train_and_predict(male_embeddings, female_embeddings):
    assert(len(male_embeddings) == len(female_embeddings))
    assert(len(male_embeddings) % 5 == 0)
    total = len(male_embeddings)
    train_size, test_size = int(total * 0.2), int(total * 0.8)
    
    # Train
    train_embed = torch.cat([male_embeddings[:train_size], female_embeddings[:train_size]], dim = 0)
    train_label = [1] * train_size + [0] * train_size

    clf = svm.SVC()
    clf.fit(train_embed.cpu(), train_label)

    # Test
    test_embed = torch.cat([male_embeddings[train_size:total], female_embeddings[train_size:total]], dim = 0)
    test_label = [1] * test_size + [0] * test_size

    predictions = clf.predict(test_embed.cpu())
    accuracy = [1 if pred == label else 0 for pred, label in zip(predictions, test_label)]
    print("Classification accuracy: ", sum(accuracy) / len(accuracy))

In [ ]:
# 2016 Hard_debiased
# Classify 2500 most biased words before and after debiasing

# debiased model with bert_words
debiased_top500_male_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_2016_male_2500_embeddings.pkl", 0)
debiased_top500_female_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_2016_female_2500_embeddings.pkl", 0)

# debiased model with debiased_words
hong_debiased_top500_male_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_hong2016_male_2500_embeddings.pkl", 0)
hong_debiased_top500_female_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_hong2016_female_2500_embeddings.pkl", 0)

# Bert
random.shuffle(debiased_top500_male_embeddings)
random.shuffle(debiased_top500_female_embeddings)
train_and_predict(debiased_top500_male_embeddings, debiased_top500_female_embeddings)

# Debiased
random.shuffle(hong_debiased_top500_male_embeddings)
random.shuffle(hong_debiased_top500_female_embeddings)
train_and_predict(hong_debiased_top500_male_embeddings, hong_debiased_top500_female_embeddings)

In [ ]:
# 2018 GN_GloVe
# Classify 2500 most biased words before and after debiasing

# debiased model with bert_words
debiased_top500_male_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_2018_male_2500_embeddings.pkl", 0)
debiased_top500_female_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_2018_female_2500_embeddings.pkl", 0)

# debiased model with debiased_words
hong_debiased_top500_male_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_hong2018_male_2500_embeddings.pkl", 0)
hong_debiased_top500_female_embeddings = open_pklfile(f"{prefix}/{debiased_folder}/{debiased_model}_{word_or_sent}_hong2018_female_2500_embeddings.pkl", 0)

# Bert
random.shuffle(debiased_top500_male_embeddings)
random.shuffle(debiased_top500_female_embeddings)
train_and_predict(debiased_top500_male_embeddings, debiased_top500_female_embeddings)

# Debiased
random.shuffle(hong_debiased_top500_male_embeddings)
random.shuffle(hong_debiased_top500_female_embeddings)
train_and_predict(hong_debiased_top500_male_embeddings, hong_debiased_top500_female_embeddings)
